# Internet Outage Live Map

## Steps

### 1 - Scrape the page -> 
### 2 - Put in on a site ->

In [32]:
# Step 1
import requests
from bs4 import BeautifulSoup

URL = "https://pulse.internetsociety.org/en/shutdowns/?country_code=&only_ongoing=on"

response = requests.get(URL)
response.raise_for_status()  # fails loudly if something is wrong

soup = BeautifulSoup(response.text, "html.parser")

# Find all shutdown cards
cards = soup.find_all("a", class_="shutdown")

print(f"Found {len(cards)} shutdown cards")

# Peek at the first one (short preview, not overwhelming)
cards[0].prettify()[:1000]


Found 10 shutdown cards


'<a class="card invert shutdown ongoing" href="/en/shutdowns/internet-suspended-in-balochistan-pakistan-august-2025/">\n <div class="flow shutdown-details">\n  <div class="lozenge ongoing">\n   <span class="chip">\n   </span>\n   Ongoing\n  </div>\n  <h2 class="type">\n   Regional shutdown\n  </h2>\n  <h3>\n   Mobile Internet\n  </h3>\n  <p class="regions h4">\n   Balochistan\n  </p>\n  <div class="overview p-summary">\n   The Pakistan Telecommunication Authority (PTA) has reportedly ordered Internet Service Providers to suspend mobile Internet services in the Balochistan region due to security concerns.\n  </div>\n </div>\n <div class="shutdown-data card dark noborder">\n  <h3 class="h2 country-name row reverse">\n   <img alt="Flag of Pakistan" class="flag square" src="/static/img/flags/full/PK.png"/>\n   <span>\n    Pakistan\n   </span>\n  </h3>\n  <ul class="list-reset incident-time">\n   <li>\n    <b>\n     Start\n    </b>\n    <time class="dt-start_date" datetime="2025-08-05T16:00

In [31]:
card = cards[0]

data = {}

# URL
data["detail_url"] = "https://pulse.internetsociety.org" + card.get("href", "")

# Status (Ongoing / Ended)
status_tag = card.find("div", class_="lozenge")
data["status"] = status_tag.get_text(strip=True) if status_tag else None

# Shutdown type
type_tag = card.find("h2", class_="type")
data["shutdown_type"] = type_tag.get_text(strip=True) if type_tag else None

# Connection type (e.g. Mobile Internet)
connection_tag = card.find("h3")
data["connection_type"] = connection_tag.get_text(strip=True) if connection_tag else None

# Region
region_tag = card.find("p", class_="regions")
data["region"] = region_tag.get_text(strip=True) if region_tag else None

# Summary
summary_tag = card.find("div", class_="overview")
data["summary"] = summary_tag.get_text(strip=True) if summary_tag else None

# Country name
country_tag = card.find("h3", class_="country-name")
data["country"] = country_tag.get_text(strip=True) if country_tag else None

# Country code (from flag image)
flag_img = card.find("img", class_="flag")
if flag_img and flag_img.get("src"):
    data["country_code"] = flag_img["src"].split("/")[-1].replace(".png", "")
else:
    data["country_code"] = None

# Start & End times
start_time = card.find("time", class_="dt-start_date")
data["start_datetime"] = start_time.get("datetime") if start_time else None

end_time = card.find("time", class_="dt-end_date")
data["end_datetime"] = end_time.get("datetime") if end_time else None

# Duration
duration_li = card.find("li", string=lambda x: x and "Total Duration" in x)
data["duration"] = duration_li.get_text(strip=True) if duration_li else None

data



{'detail_url': 'https://pulse.internetsociety.org/en/shutdowns/internet-suspended-in-balochistan-pakistan-august-2025/',
 'status': 'Ongoing',
 'shutdown_type': 'Regional shutdown',
 'connection_type': 'Mobile Internet',
 'region': 'Balochistan',
 'summary': 'The Pakistan Telecommunication Authority (PTA) has reportedly ordered Internet Service Providers to suspend mobile Internet services in the Balochistan region due to security concerns.',
 'country': 'Pakistan',
 'country_code': 'PK',
 'start_datetime': '2025-08-05T16:00:00+00:00',
 'end_datetime': '',
 'duration': None}

In [30]:
all_data = []

print(f"Starting scrape of {len(cards)} shutdowns...\n")

for i, card in enumerate(cards, start=1):
    item = {}

    item["detail_url"] = "https://pulse.internetsociety.org" + card.get("href", "")

    status_tag = card.find("div", class_="lozenge")
    item["status"] = status_tag.get_text(strip=True) if status_tag else None

    type_tag = card.find("h2", class_="type")
    item["shutdown_type"] = type_tag.get_text(strip=True) if type_tag else None

    connection_tag = card.find("h3")
    item["connection_type"] = connection_tag.get_text(strip=True) if connection_tag else None

    region_tag = card.find("p", class_="regions")
    item["region"] = region_tag.get_text(strip=True) if region_tag else None

    summary_tag = card.find("div", class_="overview")
    item["summary"] = summary_tag.get_text(strip=True) if summary_tag else None

    country_tag = card.find("h3", class_="country-name")
    item["country"] = country_tag.get_text(strip=True) if country_tag else None

    flag_img = card.find("img", class_="flag")
    if flag_img and flag_img.get("src"):
        item["country_code"] = flag_img["src"].split("/")[-1].replace(".png", "")
    else:
        item["country_code"] = None

    start_time = card.find("time", class_="dt-start_date")
    item["start_datetime"] = start_time.get("datetime") if start_time else None

    end_time = card.find("time", class_="dt-end_date")
    item["end_datetime"] = end_time.get("datetime") if end_time else None

    # CLEANED duration extraction
    duration_li = card.find(lambda tag: tag.name == "li" and tag.b and "Total Duration" in tag.b.get_text())
    if duration_li:
        # remove the "Total Duration" label and keep only the value
        item["duration"] = duration_li.get_text(strip=True).replace("Total Duration", "").strip()
    else:
        item["duration"] = None

    all_data.append(item)

    print(f"Scraped {i}/{len(cards)}")

print("\nDone.")




Starting scrape of 10 shutdowns...

Scraped 1/10
Scraped 2/10
Scraped 3/10
Scraped 4/10
Scraped 5/10
Scraped 6/10
Scraped 7/10
Scraped 8/10
Scraped 9/10
Scraped 10/10

Done.


In [12]:
all_data[:2]


[{'detail_url': 'https://pulse.internetsociety.org/en/shutdowns/internet-suspended-in-balochistan-pakistan-august-2025/',
  'status': 'Ongoing',
  'shutdown_type': 'Regional shutdown',
  'connection_type': 'Mobile Internet',
  'region': 'Balochistan',
  'summary': 'The Pakistan Telecommunication Authority (PTA) has reportedly ordered Internet Service Providers to suspend mobile Internet services in the Balochistan region due to security concerns.',
  'country': 'Pakistan',
  'country_code': 'PK',
  'start_datetime': '2025-08-05T16:00:00+00:00',
  'end_datetime': '',
  'duration': '130 days, 9 hours'},
 {'detail_url': 'https://pulse.internetsociety.org/en/shutdowns/blocking-of-services-in-togo/',
  'status': 'Ongoing',
  'shutdown_type': 'Service blocking',
  'connection_type': 'Facebook, Signal, Telegram',
  'region': None,
  'summary': 'Amid political unrest in Togo, the government is limiting access to multiple services on the three major networks of the country, Yas Togo (TogoTeleco

In [29]:
import pandas as pd

df = pd.DataFrame(all_data)

# Preview the data
df.head()

# Save to CSV
output_file = "internet_shutdowns_ongoing.csv"
df.to_csv(output_file, index=False)

print(f"Saved {len(df)} rows to '{output_file}'")


Saved 10 rows to 'internet_shutdowns_ongoing.csv'


In [33]:
df.head(20)


,detail_url,status,shutdown_type,connection_type,region,summary,country,country_code,start_datetime,end_datetime,duration
0,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,Mobile Internet,Balochistan,The Pakistan Telecommunication Authority (PTA)...,Pakistan,PK,2025-08-05T16:00:00+00:00,,"130 days, 9 hours"
1,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Service blocking,"Facebook, Signal, Telegram",None,"Amid political unrest in Togo, the government ...",Togo,TG,2025-06-26T07:00:00+00:00,,"170 days, 18 hours"
2,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,Iraq,Kurdistan,There are reports that Iraq’s Communications a...,Iraq,IQ,2025-02-18T11:00:00+00:00,,"298 days, 14 hours"
3,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,Mobile Internet,Balochistan,Local authorities have suspended mobile Intern...,Pakistan,PK,2024-11-15T00:00:00+00:00,,"394 days, 1 hour"
4,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,Equatorial Guinea,Annobón,Equatorial Guinea authorities ordered fixed an...,Equatorial Guinea,GQ,2024-07-20T00:00:00+00:00,,"512 days, 1 hour"
5,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,"Palestine, State of",Gaza Strip,Internet connectivity into Gaza started being ...,"Palestine, State of",PS,2023-10-07T14:00:00+00:00,,"798 days, 11 hours"
6,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Service blocking,"DoH, WhatsApp, Instagram, Google Play Store, A...",None,Multiple Internet services have been blocked i...,Iran,IR,2022-09-20T00:00:00+00:00,,"1181 days, 1 hour"
7,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Service blocking,"Twitter, Facebook, Instagram, News Sites",None,"On 26 February 2022, the Russian government be...",Russia,RU,2022-02-26T00:00:00+00:00,,"1387 days, 1 hour"
8,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Service blocking,"Facebook, Twitter, Instagram",None,"On 1st February 2021, the military in Myanmar ...",Myanmar,MM,2021-02-02T00:00:00+00:00,,"1776 days, 1 hour"
9,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Service blocking,"Facebook, Google, Instagram, LinkedIn, Messeng...",None,Multiple Internet services have been blocked i...,China,CN,2019-01-01T00:00:00+00:00,,"2539 days, 1 hour"


In [41]:
import pandas as pd
import re

# Load CSV
df = pd.read_csv("internet_shutdowns_ongoing.csv")

# Parse duration string into total days
def parse_duration(duration_str):
    if pd.isna(duration_str) or duration_str.strip() == "":
        return None
    match = re.match(r"(\d+)\s+days?,\s*(\d+)\s+hours?", duration_str)
    if match:
        days = int(match.group(1))
        hours = int(match.group(2))
        total_days = days + hours / 24
        return total_days
    else:
        return None

df['duration_days'] = df['duration'].apply(parse_duration)

# Round/floor duration_days to whole number
df['duration_days'] = df['duration_days'].round().astype('Int64')  # Int64 allows NaN values

# Define bins (including a bin for >2000 days)
bins = [0, 30, 90, 180, 365, 730, 2000, float('inf')]
labels = ["<1 month", "1-3 months", "3-6 months", "6-12 months", "1-2 years", "2-5 years", ">5 years"]

df['duration_category'] = pd.cut(df['duration_days'], bins=bins, labels=labels, right=False)

# Save the updated DataFrame to CSV
df.to_csv("internet_shutdowns_processed.csv", index=False)

print("CSV saved as 'internet_shutdowns_processed.csv'")

df.head()




CSV saved as 'internet_shutdowns_processed.csv'


,detail_url,status,shutdown_type,connection_type,region,summary,country,country_code,start_datetime,end_datetime,duration,duration_days,duration_category
0,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,Mobile Internet,Balochistan,The Pakistan Telecommunication Authority (PTA)...,Pakistan,PK,2025-08-05T16:00:00+00:00,NaN,"130 days, 9 hours",130,3-6 months
1,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Service blocking,"Facebook, Signal, Telegram",NaN,"Amid political unrest in Togo, the government ...",Togo,TG,2025-06-26T07:00:00+00:00,NaN,"170 days, 18 hours",171,3-6 months
2,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,Iraq,Kurdistan,There are reports that Iraq’s Communications a...,Iraq,IQ,2025-02-18T11:00:00+00:00,NaN,"298 days, 14 hours",299,6-12 months
3,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,Mobile Internet,Balochistan,Local authorities have suspended mobile Intern...,Pakistan,PK,2024-11-15T00:00:00+00:00,NaN,"394 days, 1 hour",394,1-2 years
4,https://pulse.internetsociety.org/en/shutdowns...,Ongoing,Regional shutdown,Equatorial Guinea,Annobón,Equatorial Guinea authorities ordered fixed an...,Equatorial Guinea,GQ,2024-07-20T00:00:00+00:00,NaN,"512 days, 1 hour",512,1-2 years
